# K Means Clustering

K-means clustering adalah algoritma pembelajaran sederhana yang digunakan untuk memecahkan masalah clustering. Algoritma ini mengikuti prosedur sederhana untuk mengklasifikasikan kumpulan data yang diberikan ke dalam sejumlah cluster, yang ditentukan sebelumnya. lalu bagaiamana model basis dari k-means

- Setiap cluster memiliki titik pusat yang merupakan rata-rata dari semua titik yang terdapat di dalam cluster tersebut
- Setiap titik lebih dekat ke titik pusat clusternya daripada ke titik pusat cluster lainnya

### Cara Kerja K Means
  Inisialisasi : Pilih k titik acak sebagai pusat cluster (centroid)  

  Iterasi:  
    - Hitung jarak setiap titik ke centroid   
    - Tentukan cluster dari titik tersebut berdasarkan jarak terdekat   
    - Ubah centroid dengan mengambil rata-rata dari titik yang berada di cluster yang sama 

  Berhenti berdasarkan kriteria berikut:  
    - Tidak ada perubahan cluster   
    - Jumlah iterasi sudah mencapai batas maksimum

Cara menghitung jarak tiap titik ke centroid  
  - Jarak euclidean  
  - Jarak manhattan  

Kali ini kita menggunakan jarak euclidean untuk menghitung jarak tiap titik ke centroidnya

In [18]:
def euclidean_distance(point_1: list, point_2: list) -> float:
    return sum([(a - b) ** 2 for a, b in zip(point_1, point_2)]) ** 0.5

Dan untuk menghitung jarak terdekat di antara  titik dengan centroid, kita dapat menggunakan fungsi berikut

In [19]:
def closest_point(point: list, centroids: list) -> list: 
    return min(centroids, key=lambda centroid: euclidean_distance(point, centroid))

Lalu kita akan menghitung rata-rata dari titik yang berada di cluster yang sama

In [20]:
def mean(points: list) -> list:
    return [sum(x) / len(x) for x in zip(*points)]

Setelah semua fungsi di atas sudah kita buat, kita akan membuat implementasi kelas KMeans

In [21]:
import random
class KMeans:
    def __init__(self, data: list, n_clusters=2, max_iteration=300, n_init=10):
        self.n_clusters = n_clusters
        self.data = data
        self.max_iteration = max_iteration
        self.n_init = n_init

        self.centroids = []
        self.clusters = []
        self.inertia = None

    def kmeans(self):
        centroids = random.sample(self.data, self.n_clusters)
        for _ in range(self.max_iteration):
            clusters = [[] for _ in range(self.n_clusters)]
            for x in self.data:
                closest = closest_point(x, centroids)
                clusters[centroids.index(closest)].append(x)
            new_centroids = []
            for cluster in clusters:
                new_centroids.append(mean(cluster))
            if new_centroids == centroids:
                break
            centroids = new_centroids
        return centroids, clusters

Dikarenakan pengambilan centroid awal dilakukan secara random, hasil clustering yang didapatkan tidak akan selalu optimal. 

In [ ]:
for i in range(10):
    kmeans = KMeans([[1, 2], [1, 4], [10, 2], [10, 4]])
    centroids, clusters = kmeans.kmeans()
    print(f"Iteration {i}: {centroids}")

Oleh karena itu kita akan mengulang proses clustering sebanyak n_init kali dan menghitung nilai inertia untuk setiap cluster yang dihasilkan. Lalu kita akan mengambil cluster dengan nilai inertia terkecil

In [ ]:
def fit(self):
    for _ in range(self.n_init):
        centroids, clusters = self.kmeans()

        current_inertia = 0
        for centroid, cluster in zip(centroids, clusters):
            for point in cluster:
                current_inertia += euclidean_distance(point, centroid) ** 2
        if self.inertia is None or current_inertia < self.inertia:
            self.inertia = current_inertia
            self.centroids = centroids
            self.clusters = clusters
            
KMeans.fit = fit

for i in range(5):
    kmeans = KMeans([[1, 2], [1, 4], [10, 2], [10, 4]])
    kmeans.fit()
    print(kmeans.centroids)

Sebelum melakukan percobaan, kita akan mengimport library dan fungsi yang dibutuhkan

In [32]:
def generate_random_data(n=100, k=2, min=0, max=100) -> list:
    return [[random.randint(min, max) for _ in range(k)] for _ in range(n)]

import matplotlib.pyplot as plt

def plot_2d(data: list, centroids: list):
    colors = ['r', 'y', 'b', 'c', 'k', 'g', 'm']
    for i, centroid in enumerate(centroids):
        plt.scatter([x[0] for x in data[i]], [x[1]
                                              for x in data[i]], c=colors[i])
        plt.scatter(centroid[0], centroid[1], c='black', marker='x')

    plt.show()


def plot_3d(clusters: list, centroids: list):
    colors = ['r', 'y', 'b', 'c', 'k', 'g', 'm']
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    for i, cluster in enumerate(clusters):
        ax.scatter([x[0] for x in cluster], [x[1]
                                             for x in cluster], [x[2] for x in cluster], c=colors[i])
        ax.scatter(centroids[i][0], centroids[i][1],
                   centroids[i][2], c='black', marker='x')

Uji coba k-means clustering dengan dataset acak

In [ ]:
data = generate_random_data(n=100, k=2)
kmeans = KMeans(data, n_clusters=3)
kmeans.fit()
# print(kmeans.centroids)
plot_3d(kmeans.clusters, kmeans.centroids)